# Setup

In [ ]:
install_load <- function (package1, ...)  {   

   # convert arguments to vector
   packages <- c(package1, ...)

   # start loop to determine if each package is installed
   for(package in packages){

       # if package is installed locally, load
       if(package %in% rownames(installed.packages()))
          do.call('library', list(package))

       # if package is not installed locally, download, then load
       else {
          install.packages(package)
          do.call("library", list(package))
       }
   } 
}

## Section 11.6

In [ ]:
install_load("tidyverse", "parallel", "parallelMap", "mlr")

### Listing 11.1

In [ ]:
data(Iowa, package = "lasso2")
iowaTib <- as_tibble(Iowa)
iowaTib

### Listing 11.2

In [ ]:
iowUntidy <- gather(iowaTib, "Variable", "Value", -Yield)

options(repr.plot.width = 12, repr.plot.height = 8)
ggplot(iowaUntidy, aes(Value, Yield)) +
    facet_wrap(~Variable, scales = "free_x") + 
    geom_point() +
    geom_smooth(method = "lm") +
    theme_bw()

## Section 11.6.2

### Listing 11.3

In [ ]:
iowaTask <- makeRegrTask(data = iowaTib, target = "Yield")
ridge <- makeLearner("regr.glmnet", alpha = 0, id = "ridge")

### Listing 11.4

In [ ]:
filterVals <- generateFilterValuesData(iowaTask)
plotFilterValues(filterVals) + theme_bw()

### Listing 11.5

In [ ]:
ridgeParamSpace <- makeParamSet(
    nameNumericParam("s", lower = 0, upper = 15))
randSearch <- makeTuneControlRandom(maxit = 200)
cvForTuning <- makeResampleDesc("RepCV", folds = 3, reps = 10)

parallelStartSocket(cpus = detectCores())
tunedRidgePars <- tuneParmas(ridge, task = iowaTask,
                            resampling = cvForTuning,
                            par.set = ridgeParamSpace,
                            control = randSearch)
parallelStop()

tunedRidgePars

### Listing 11.6

In [ ]:
ridgeTuningData <- generateHyperParsEffectData(tunedRidgePars)
plotHyperParsEffect(ridgeTuningData, x = "s", y = "mse.test.mena", plot.type = "line") +
    theme_bw()

### Exercise 1

In [ ]:
# update search to 50
ridgeParamSpace50 <- makeParamSet(
  makeNumericParam("s", lower = 0, upper = 50))

parallelStartSocket(cpus = detectCores())
tunedRidgePars50 <- tuneParams(ridge, task = iowaTask,
                             resampling = cvForTuning,
                             par.set = ridgeParamSpace50,
                             control = randSearch)
parallelStop()

ridgeTuningData50 <- generateHyperParsEffectData(tunedRidgePars50)
plotHyperParsEffect(ridgeTuningData50, x = "s", y = "mse.test.mean",
                    plot.type = "line") +
  theme_bw()

### Listing 11.7

In [ ]:
tunedRidge <- setHyperPars(ridge, par.vals = tunedRidgePars$x)
tunedRidgeModel <- train(tunedRidge, iowaTask)

### Listing 11.8

In [ ]:
ridgeModelData <- getLearnerModel(tunedRidgeModel)
ridgeCoefs <- coef(ridgeModeldata, s = tunedRidgePars$x$s)
ridgeCoefs

### Listing 11.9

In [ ]:
lmCoefs <- coef(lm(Yield ~ ., data = iowaTib))
coefTib <- tibble(Coef = rownames(ridgeCoefs)[-1],
                 Ridge = as.vector(ridgeCoefs)[-1],
                 Lm = as.vector(lmCoefs)[-1]))
coefUntidy <- gather(coefTib, key = Model, value = Beta, -Coef)
ggplot(coefUntidy, aes(reorder(Coef, Beta), Beta, fill = Model)) + 
    geom_bar(stat = "identity", col = "black") +
    facet_wrap(~Model) +
    theme_bw() +
    theme(legend.position = "none")

### Exercise 2

In [ ]:
coefTib2 <- tibble(Coef = rownames(ridgeCoefs),
                  Ridge = as.vector(ridgeCoefs),
                  Lm = as.vector(lmCoefs)))
coefUntidy2 <- gather(coefTib2, key = Model, value = Beta, -Coef)
ggplot(coefUntidy2, aes(reorder(Coef, Beta), Beta, fill = Model)) + 
    geom_bar(stat = "identity", col = "black") +
    facet_wrap(~Model) +
    theme_bw() +
    theme(legend.position = "none")